# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os as csv
import gmaps
import json
from pprint import pprint

# Import API key
from api_keys import g_key
url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
path = 'output/city_weather.csv'
csvfile = pd.read_csv(path)
df = pd.DataFrame(csvfile)
df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,cape town,-36.41,-71.90,66.20,45,63,9.17,CL,1600807465
1,kapaa,22.08,-159.32,82.99,65,75,10.29,US,1600807466
2,castro,-24.79,-50.01,57.52,73,41,9.95,BR,1600807467
3,ponta do sol,38.73,-28.27,69.80,73,40,16.11,PT,1600807468
4,leningradskiy,69.38,178.42,39.52,83,100,17.22,RU,1600807469
...,...,...,...,...,...,...,...,...,...
535,charters towers,38.57,-83.43,69.80,40,1,5.82,US,1600808086
536,odda,60.07,6.55,57.99,80,99,18.01,NO,1600808041
537,burnie,-41.07,145.92,51.01,83,87,11.99,AU,1600808088
538,muros,42.78,-9.06,64.99,92,97,8.99,ES,1600808089


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key=g_key)

In [5]:
locations = df[["Lat", "Lng"]].astype(float)
weight = df["Humidity"].astype(float)

In [6]:
fig = gmaps.figure()
heatmap_layer = gmaps.heatmap_layer(locations, weights=weight, dissipating=False, 
                                    max_intensity=100, point_radius=1)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
criteria1= df[df["Max Temp"] >= 70]
criteria2 = criteria1[criteria1["Max Temp"] <= 85]
criteria3 = criteria2[criteria2["Wind Speed"] < 10]
hotel_df = criteria3[criteria3["Cloudiness"] == 0]
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
27,beroroha,-21.67,45.17,72.52,58,0,4.21,MG,1600807496
103,jalu,29.03,21.55,76.73,37,0,7.78,LY,1600807585
131,richards bay,46.84,-95.91,80.60,24,0,4.70,US,1600807618
151,fereydun kenar,34.37,46.92,73.40,15,0,6.93,IR,1600807642
226,boende,-0.22,20.87,73.17,92,0,3.02,CD,1600807729
332,riyadh,24.69,46.72,84.20,16,0,4.70,SA,1600807670
370,poum,-20.23,164.02,71.47,73,0,7.63,NC,1600807897


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
hotel_df["Hotel Name"] = ''
hotel_df.reset_index()

C:\Users\Babyta\anaconda3\envs\pythondata\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,index,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,27,beroroha,-21.67,45.17,72.52,58,0,4.21,MG,1600807496,
1,103,jalu,29.03,21.55,76.73,37,0,7.78,LY,1600807585,
2,131,richards bay,46.84,-95.91,80.60,24,0,4.70,US,1600807618,
3,151,fereydun kenar,34.37,46.92,73.40,15,0,6.93,IR,1600807642,
4,226,boende,-0.22,20.87,73.17,92,0,3.02,CD,1600807729,
5,332,riyadh,24.69,46.72,84.20,16,0,4.70,SA,1600807670,
6,370,poum,-20.23,164.02,71.47,73,0,7.63,NC,1600807897,


In [9]:
params = {'radius':5000,'type': "lodging", 'key': g_key}
for index, row in hotel_df.iterrows():
    lat = row['Lat']
    long = row['Lng']
    params['location'] = f'{lat},{long}'
    response = requests.get(url, params=params)
    hotel_name = response.json()
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name['results'][0]['name']
    except (KeyError, IndexError,ValueError):
        print("No hotel found/Skipping...")

No hotel found/Skipping...


C:\Users\Babyta\anaconda3\envs\pythondata\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


No hotel found/Skipping...
No hotel found/Skipping...


In [14]:
params = {'radius':5000,'type': "lodging", 'key': g_key}
for index, row in hotel_df.iterrows():
    lat = row['Lat']
    long = row['Lng']
    params['location'] = f'{lat},{long}'
    response = requests.get(url, params=params)
    hotel_name = response.json()
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name['results'][0]['name']
    except (KeyError, IndexError,ValueError):
        print("No hotel found/Skipping...")
#         nan = "NaN"
#         hotel_df.replace("", np.nan, inplace=True)
#         hotel_df.fillna("", inplace=False)

AttributeError: 'NoneType' object has no attribute 'iterrows'

In [15]:
# hotel_df = hotel_df.dropna(subset=["Hotel Name"])
hotel_df

In [94]:
# json.dumps(hotel_name['results'], indent=4, sort_keys=True)
pprint(hotel_name['results'])

[{'business_status': 'OPERATIONAL',
  'geometry': {'location': {'lat': -20.2317711, 'lng': 164.0230586},
               'viewport': {'northeast': {'lat': -20.2304956197085,
                                          'lng': 164.0244584802915},
                            'southwest': {'lat': -20.2331935802915,
                                          'lng': 164.0217605197085}}},
  'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png',
  'name': 'Gîte kajeon',
  'place_id': 'ChIJbzORGwTPmG4RRP1dw6fv2i4',
  'plus_code': {'compound_code': 'Q29F+76 Poum, New Caledonia',
                'global_code': '5VF6Q29F+76'},
  'rating': 3,
  'reference': 'ChIJbzORGwTPmG4RRP1dw6fv2i4',
  'scope': 'GOOGLE',
  'types': ['lodging', 'point_of_interest', 'establishment'],
  'user_ratings_total': 1,
  'vicinity': 'route de Boat Pass, CP 98826, Poum'},
 {'business_status': 'OPERATIONAL',
  'geometry': {'location': {'lat': -20.2341689, 'lng': 164.0434963},
               'viewport': {'no

In [92]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
